In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from keras.datasets import cifar10

In [ ]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
Tue Aug  4 19:44:51 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+

## Load Cifar10 Dataset

In [ ]:
(x1, y1), (x2, y2) = cifar10.load_data()

170500096/170498071 [==============================] - 4s 0us/step


## Combine test and train data

In [ ]:
x = np.concatenate((x1, x2), axis=0)
y = np.concatenate((y1, y2), axis=0)

In [ ]:
del x1, y1, x2, y2

In [ ]:
import time
import matplotlib.pyplot as plt
import keras
from keras.preprocessing import image
from keras.models import Model
from keras.layers import *
from keras.optimizers import Adam, RMSprop
from keras.layers.advanced_activations import LeakyReLU

Batchsize of 50 10 class, noise dim is 2048

In [ ]:
img_size = 32
noise_size = 2048
batch_size = 50
classes = 10

## Define Generator model
in generator we have we have noise as input layer and embed layer and classes of noise size while flatening them</br>
then we have a 2048 fully connected layer for getting noise (noise dim is 2048) </br>
then there are 4 Conv transpose or deConv layers to generate image at output

In [ ]:
def generator():
    
    noise = Input(shape=(noise_size, ))
    label = Input(shape=(1, ))
    
    label_embedding = Flatten()(Embedding(classes, noise_size)(label))
    
    model_input = multiply([noise, label_embedding])
    
    x = Dense(2048)(model_input)
    
    x = Reshape((2, 2, 512))(x)
    x = BatchNormalization(momentum=0.9)(x)
    x = LeakyReLU(0.1)(x)
    
    x = Conv2DTranspose(256, (5, 5), padding='same', strides=2)(x)
    x = BatchNormalization(momentum=0.9)(x)
    x = LeakyReLU(0.1)(x)
    
    x = Conv2DTranspose(128, (5, 5), padding='same', strides=2)(x)
    x = BatchNormalization(momentum=0.9)(x)
    x = LeakyReLU(0.1)(x)
    
    x = Conv2DTranspose(64, (5, 5), padding='same', strides=2)(x)
    x = BatchNormalization(momentum=0.9)(x)
    x = LeakyReLU(0.1)(x)
    
    x = Conv2DTranspose(3, (5, 5), padding='same', strides=2)(x)
    img = Activation('tanh')(x)
    
    return Model([noise, label], img)

## Define discriminator Network
here we get image that created by generator as input at 3 conv layer and 1 label input while flatening to poduce label output in CGAN </br>
then 1 fully connected layer after dropout to produce real or fakeness

In [ ]:
def discriminator():
    
    img = Input(shape=(img_size, img_size, 3))
    
    x = GaussianNoise(0.1)(img)
    
    x = Conv2D(64, (3, 3), padding='same', strides = 2)(x)
    x = BatchNormalization(momentum=0.9)(x)
    x = LeakyReLU(0.2)(x)
        
    x = Conv2D(128, (3, 3), padding='same', strides = 2)(x)
    x = BatchNormalization(momentum=0.9)(x)
    x = LeakyReLU(0.2)(x)
        
    x = Conv2D(256, (3, 3), padding='same', strides = 2)(x)
    x = BatchNormalization(momentum=0.9)(x)
    x = LeakyReLU(0.2)(x)
        
    x = Conv2D(512, (3, 3), padding='same', strides = 2)(x)
    x = BatchNormalization(momentum=0.9)(x)
    x = LeakyReLU(0.2)(x)
    
    label = Input(shape=(1, ))
    label_embedding = Flatten()(Embedding(classes, noise_size)(label))
    
    flat_img = Flatten()(x)
    
    model_input = multiply([flat_img, label_embedding])

    nn = Dropout(0.3)(model_input)
    
    validity = Dense(1, activation='sigmoid')(nn)
    
    return Model([img, label], validity)

## DEfine CGAN model
we combine generator and discriminator model with ADAM optimizer  to produce CGAN network

In [ ]:
d_model = discriminator()
d_model.compile(loss=['binary_crossentropy'], optimizer=Adam(lr=0.0002, beta_1=0.5))
d_model.trainable = False
g_model = generator()

noise = keras.Input(shape=(noise_size, ))
label = keras.Input(shape=(1, ))
img = g_model([noise, label])

valid = d_model([img, label])

combined = Model([noise, label], valid)
combined.compile(loss=['binary_crossentropy'], optimizer=Adam(lr=0.001, beta_1=0.5))

## Train Network
here we train network, for eacch batch of input and for each image in that we first produce a real and fake image and giving it to discriminator to train then by adding noise from batchsize we generate image and train generator on thatand for every 50 we print losses

In [ ]:
def train(epochs):
    d_loss_history = []
    g_loss_history  = [] 
    for epoch in range(epochs):
        
        random = np.random.randint(0, 11)
        
        for index in range(int(x.shape[0]/batch_size)):
                     
            valid = np.ones((batch_size, 1)) - (np.random.random()*0.1)
            fake = np.zeros((batch_size, 1)) + (np.random.random()*0.1)
            
            x_train = x[index*batch_size : (index+1)*batch_size]
            y_train = y[index*batch_size : (index+1)*batch_size]
            x_train = (x_train - 127.5)/127.5
            
            if index % 100 == random:
                valid = np.zeros((batch_size, 1)) + (np.random.random()*0.1)
                fake = np.ones((batch_size, 1)) - (np.random.random()*0.1)
            
            noise = np.random.randn(batch_size, noise_size)
            gen_img = g_model.predict([noise, y_train])
                        
            d_loss_real = d_model.train_on_batch([x_train, y_train], valid)
            d_loss_fake = d_model.train_on_batch([gen_img, y_train], fake)
            d_loss = 0.5*(np.add(d_loss_real, d_loss_fake))

            sample_label = np.random.randint(0, 10, batch_size).reshape(-1, 1)

            valid = np.ones((batch_size, 1))
            
            g_loss = combined.train_on_batch([noise, sample_label], valid)

            if index % (batch_size) == 0:
                print(index)
                print("%d [D loss: %f] [G loss: %f]" % (epoch, d_loss, g_loss))
                d_loss_history.append(d_loss)
                g_loss_history.append(g_loss)
                sample_images(epoch)
        
        name = './weights/combined_' + str(epoch) + '.h5'
        combined.save_weights(name)
        
        time.sleep(30)
    return g_loss_history, d_loss_history

## Sample images to Draw digures


In [ ]:
def sample_images(epoch):
    r = 2
    c = 5
    noise = np.random.randn(10, noise_size)
    sample_label = np.arange(0, 10).reshape(-1, 1)
            
    gen_img = g_model.predict([noise, sample_label])
        
    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            img = image.array_to_img(gen_img[cnt])
            axs[i,j].imshow(img)
            axs[i,j].set_title("Class: %d" % sample_label[cnt])
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/%d.png" % epoch)
    plt.close()

In [ ]:
g_loss_history, d_loss_history = train(20)

0
0 [D loss: 0.697392] [G loss: 0.692948]
50
0 [D loss: 0.578080] [G loss: 0.686301]
100
0 [D loss: 0.226013] [G loss: 0.190735]
150
0 [D loss: 0.178096] [G loss: 0.077708]
200
0 [D loss: 0.210440] [G loss: 0.046639]
250
0 [D loss: 0.140536] [G loss: 0.024208]
300
0 [D loss: 0.282827] [G loss: 0.018577]
350
0 [D loss: 0.209517] [G loss: 0.036820]
400
0 [D loss: 0.180378] [G loss: 0.022412]
450
0 [D loss: 0.288181] [G loss: 0.033512]
500
0 [D loss: 0.138622] [G loss: 0.021673]
550
0 [D loss: 0.104300] [G loss: 0.033136]
600
0 [D loss: 0.100721] [G loss: 0.023546]
650
0 [D loss: 0.085697] [G loss: 0.026715]
700
0 [D loss: 0.253280] [G loss: 0.024928]
750
0 [D loss: 0.198241] [G loss: 0.057576]
800
0 [D loss: 0.304871] [G loss: 0.028156]
850
0 [D loss: 0.175288] [G loss: 0.043680]
900
0 [D loss: 0.137001] [G loss: 0.112556]
950
0 [D loss: 0.287453] [G loss: 0.064270]
1000
0 [D loss: 0.158131] [G loss: 0.106609]
1050
0 [D loss: 0.166633] [G loss: 0.088714]
1100
0 [D loss: 0.256249] [G loss

In [ ]:
def plot_parameter(input, title)
  plt.plot(input)
  plt.title(title)
  plt.xlabel('Epoch')
  plt.ylabel('loss')
  plt.show()
plt.plot(d_loss_history)
plt.plot(g_loss_history)